In [13]:
from common import *

In [3]:
@dataclass
class RawMovie:
    name: str
    url: str
@dataclass
class Screening:
    date: datetime
    theatre: str
@dataclass
class Movie(RawMovie):
    screening: list[Screening]

In [4]:
url = "https://hkmovie6.com/showing"
headers = {
    'Cookie': 'lang=enGB;'
}

# Send a GET request to the URL
response = requests.get(url,headers=headers)
if response.status_code != 200:
    print(f"Failed to fetch the page. Status code: {response.status_code}")
soup = BeautifulSoup(response.text, 'html.parser')

# Find all div elements with class starting with 'name'
movie_name_divs = soup.find_all('div', class_=lambda x: x and x.startswith('movie show'))

RawMovies: list[RawMovie] = []

for name_div in movie_name_divs:
  href = name_div.find_next('a')['href'] if name_div.find('a') else None
  name = name_div.get_text().split('\n')[1].strip()
  print(f"{name} - {href}")
  RawMovies.append(RawMovie(name=name,url=href))

𝗺𝟲 𝘀𝗰𝗿𝗲𝗲𝗻𝗶𝗻𝗴 𝟮𝟴 ➤ HERETIC Special Screening - /movie/1ecacb13-2900-461d-9ac2-04083fe26d4d/%F0%9D%97%BA%F0%9D%9F%B2_%F0%9D%98%80%F0%9D%97%B0%F0%9D%97%BF%F0%9D%97%B2%F0%9D%97%B2%F0%9D%97%BB%F0%9D%97%B6%F0%9D%97%BB%F0%9D%97%B4_%F0%9D%9F%AE%F0%9D%9F%B4_%E2%9E%A4_HERETIC_Special_Screening
Ne Zha 2 - /movie/734ed833-9b24-4b65-8bab-4104a77e8647/Ne_Zha_2
Captain America: Brave New World - /movie/c1841576-6760-4e75-8a2a-ed19000886de/Captain_America:_Brave_New_World
Concert Information - https://hkmovie6.com/filmFest/a59c3110-41b0-444a-82fe-80fe8bc925f1
THE WAY WE TALK - /movie/90ad6c1a-2ffe-4304-9dd5-27594fb58176/THE_WAY_WE_TALK
The King Prawn Series - https://www.doughbros.com/hk-movie
Detective Chinatown 1900 - /movie/22d00c47-6179-4d0e-8bef-3d8974a8c1e7/Detective_Chinatown_1900
Forbidden Fairytale - /movie/6961e0b1-1d6e-4a88-8713-e0811a5b8c27/Forbidden_Fairytale
A Complete Unknown - /movie/06958c19-aadd-4797-93e3-9c61a06b288b/A_Complete_Unknown
Maria - /movie/ae50b185-6381-4947-b68b-92427ed4

In [5]:
chrome_options = Options() 
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
now = datetime.now()

movies_today = []
driver = webdriver.Chrome(options=chrome_options)
for rawMovie in RawMovies:
  url = "https://hkmovie6.com" + rawMovie.url + "/showtime"
  try:
    driver.get(url)
  except:
    continue
  date = driver.find_element(By.CLASS_NAME, "date").text
  (day, month) = date.split("/")
  if not now.month == int(month) or not now.day == int(day):
    print("Not today")
    continue
  cinemas_list = driver.find_elements(By.CLASS_NAME, "cinemas")
  if len(cinemas_list) == 0:
    continue
  cinemas_divs = driver.find_elements(By.CLASS_NAME, "cinemas")[0].find_elements(By.CLASS_NAME, "cinema")
  screenings = []
  for cinemas_div in cinemas_divs:
    theatre = cinemas_div.find_element(By.CLASS_NAME, "cinemaName").text
    time_divs = cinemas_div.find_elements(By.CLASS_NAME, "time")
    for time_div in time_divs:
      try:
        if len(time_div.text.split(":")) == 0:
          continue 
        screening = Screening(date=now.replace(hour=int(time_div.text.split(":")[0]),minute=int(time_div.text.split(":")[1]),second=0),theatre=theatre)
        screenings.append(screening)
      except:
        continue
  movie = Movie(name=rawMovie.name,url=rawMovie.url,screening=screenings)
  movies_today.append(movie)
  print(f"{movie.name} - {movie.screening}")

driver.quit()

Not today
Captain America: Brave New World - [Screening(date=datetime.datetime(2025, 2, 23, 22, 15, 0, 815098), theatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.datetime(2025, 2, 23, 9, 5, 0, 815098), theatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.datetime(2025, 2, 23, 11, 20, 0, 815098), theatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.datetime(2025, 2, 23, 17, 15, 0, 815098), theatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.datetime(2025, 2, 23, 19, 30, 0, 815098), theatre='英皇戲院（銅鑼灣時代廣場）'), Screening(date=datetime.datetime(2025, 2, 23, 9, 55, 0, 815098), theatre='英皇戲院（中環娛樂行）'), Screening(date=datetime.datetime(2025, 2, 23, 12, 10, 0, 815098), theatre='英皇戲院（中環娛樂行）'), Screening(date=datetime.datetime(2025, 2, 23, 19, 40, 0, 815098), theatre='英皇戲院（中環娛樂行）'), Screening(date=datetime.datetime(2025, 2, 23, 21, 55, 0, 815098), theatre='英皇戲院（中環娛樂行）'), Screening(date=datetime.datetime(2025, 2, 23, 22, 20, 0, 815098), theatre='英皇戲院（中環娛樂行）'), Screening(date=datetime.datetime(2025, 2, 23, 9

In [22]:
movies_today_df = pd.DataFrame(movies_today)
movies_today_df.to_csv(f"movies_today_{now.year}_{now.month}_{now.day}.csv",index=False)